In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
import json

import torch

import utils
import utils.configurable as conf

In [121]:
class SubModule(conf.Configurable, torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.W = torch.nn.Parameter(torch.ones(2,2))
        self.x = conf.Parameter(dtype=int)
        self.a = conf.Parameter([0,1,2.1])
        
class Module(conf.Configurable, torch.nn.Module):
    def __init__(self, y, z):
        super().__init__()
        self.sub_module = SubModule()
        self.y = y
        self.z = z

m = Module(conf.Parameter([1,2]), torch.nn.Parameter(torch.zeros(2,2)))
config_dict = m.config_dict()
print(config_dict)
config_dict['sub_module.a'][0] = 5.5
m.load_config_dict(config_dict)
config_dict = m.config_dict()
print(config_dict)
print(m.y)

{'y': [1, 2], 'sub_module.x': 0, 'sub_module.a': [0, 1, 2.1]}
{'y': [1, 2], 'sub_module.x': 0, 'sub_module.a': [5.5, 1, 2.1]}
[1, 2]


In [115]:
class A:
    def __init__(self):
        print("HERE")
        self.x = 1
    
    def __setattr__(self, name, value):
        print("in super")
        super().__setattr__(name, value)
        
class B(A):
    def __setattr__(self, name, value):
        print("in child")
        super().__setattr__(name, value)
        
b = B()

HERE
in child
in super


In [119]:
class A(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.W = torch.nn.Linear(2,2)
        self.x = torch.nn.Parameter(torch.tensor(2.0))
        
a = A()
# a.W = 3.0
a.x = 6.0

TypeError: cannot assign 'float' as parameter 'x' (torch.nn.Parameter or None expected)

In [67]:
hi = conf.Parameter(torch.tensor(10))
print(hi.data, hi.dtype)

InvalidConfigParameter: data 10 is not json serializable, try converting it to a compatible object type. If not possible, you can set is_jsonable=False, but then you can't save the config dict as json.

In [72]:
class Hi(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.x = torch.nn.Linear(2,2)
        
hi = Hi()
state_dict = hi.state_dict()
print(state_dict)
state_dict['x.weight'][0,1] = 1.0
print(hi.x.weight)
hi.load_state_dict(state_dict)
print(hi.x.weight)
state_dict['x.weight'][0,1] = -2.0
print(hi.x.weight)

OrderedDict([('x.weight', tensor([[ 0.5398, -0.6444],
        [ 0.4426,  0.5372]])), ('x.bias', tensor([-0.1677, -0.0337]))])
Parameter containing:
tensor([[0.5398, 1.0000],
        [0.4426, 0.5372]], requires_grad=True)
Parameter containing:
tensor([[0.5398, 1.0000],
        [0.4426, 0.5372]], requires_grad=True)
Parameter containing:
tensor([[ 0.5398, -2.0000],
        [ 0.4426,  0.5372]], requires_grad=True)


In [84]:
hi = {'a': 1}
print('a' in hi.keys())

True
